In [ ]:
import os
import random

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns


from scipy import signal
from scipy.fft import fft,fftfreq
from scipy.fft import fftshift

import tensorflow as tf
import keras

from keras.models import Sequential,load_model
from keras.layers import Dense, Conv2D ,LSTM, MaxPooling2D , Flatten , Dropout , BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau


from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
data1_1 = pd.read_csv('../input/deapdata/(S01)/Raw EEG Data/.csv format/S01G1AllRawChannels.csv')
data2_1 = pd.read_csv('../input/deapdata/(S01)/Raw EEG Data/.csv format/S01G2AllRawChannels.csv')
data3_1 = pd.read_csv('../input/deapdata/(S01)/Raw EEG Data/.csv format/S01G3AllRawChannels.csv')
data4_1 = pd.read_csv('../input/deapdata/(S01)/Raw EEG Data/.csv format/S01G4AllRawChannels.csv')


data1_1.insert(14, "label", 0)
data2_1.insert(14, "label", 1)
data3_1.insert(14, "label", 2)
data4_1.insert(14, "label", 3)

data1_1.drop(['Unnamed: 14'], axis=1, inplace=True)
data2_1.drop(['Unnamed: 14'], axis=1, inplace=True)
data3_1.drop(['Unnamed: 14'], axis=1, inplace=True) 
data4_1.drop(['Unnamed: 14'], axis=1, inplace=True) 

sv= pd.concat([data1_1,
               data2_1,
               data3_1,
               data4_1,])
sv=sv.sample(frac=1)

sv

In [ ]:
data=sv.to_numpy()
def get_train():
    seq = data
    #seq = array(seq)
    X,y = seq[:,0:14], seq[:,14]
    #y=labels=np.argmax(y, axis=1)
    X = X.reshape((len(X),14, 1))
    print(X.shape,y.shape,X.dtype)
    return X,y

In [ ]:
X,y = get_train()
print(pd.DataFrame(y))

In [ ]:
from keras.utils import np_utils
n_classes =4
Y_t = np_utils.to_categorical(y, n_classes)
Y_t



In [ ]:
train_x, test_x, train_y, test_y = train_test_split(X, Y_t, test_size=0.2,stratify =y)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

    # define model

cnn = Sequential()

cnn.add(Conv1D(64, kernel_size = (3) ,activation='relu', input_shape=(14,1) ))#0.72
cnn.add(Conv1D(64,  kernel_size = (2) ,activation='relu', input_shape=(14,1) ))#0.7103

cnn.add(Flatten())

cnn.add(Dense(64, activation = "relu"))#accuracy: 0.7137
cnn.add(Dense(4, activation='softmax'))

adam = Adam(lr = 0.0001)
cnn.compile(optimizer='adam', 
              loss     = "categorical_crossentropy", 
              metrics  = ['accuracy'])
    
cnn.summary()

cnn.fit(train_x, train_y,batch_size=32,  epochs=10, verbose=1, 
               validation_data=(test_x, test_y))
cnn.save('cnn.h5')


In [ ]:
def cnn_model():  
    model = Sequential()
    
    model.add(Conv1D(64, kernel_size = (8) ,activation='relu', input_shape=(14,1)))
    #model.add(MaxPooling1D(pool_size = (1)))
    #model.add(Dropout(0.2))
    model.add(BatchNormalization())
    
    model.add(Conv1D(64, kernel_size = (4) ,activation='relu',))
    #model.add(MaxPooling1D(pool_size = (1)))
    #model.add(Dropout(0.2))
    model.add(BatchNormalization())
    
    model.add(Conv1D(64, kernel_size = (2) ,activation='relu' ))
    model.add(Dropout(0.1))
    model.add(MaxPooling1D(pool_size = (1)))
    
    model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dense(64, activation = "relu"))
    
    model.add(Dense(4, activation = "softmax"))
    
    adam = Adam(lr = 0.0001)
    
    model.compile(optimizer = adam , loss = "categorical_crossentropy", 
                  metrics = ['accuracy'])
    
    return model
model = cnn_model()
model.summary()
#keras.utils.plot_model(model, show_shapes=True)

In [ ]:
import seaborn as sns

#set early stopping criteria
pat = 1
n_folds=5
epochs=10
batch_size=32


early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)
model_checkpoint = ModelCheckpoint('subjek1CNN.h5', verbose=1, save_best_only=True)


def fit_and_evaluate(t_x, val_x, t_y, val_y, EPOCHS=epochs, BATCH_SIZE=batch_size):
    model = None
    model = cnn_model() 
    results = model.fit(t_x, t_y, epochs=EPOCHS, batch_size=BATCH_SIZE, 
                        callbacks=[early_stopping, model_checkpoint],validation_split=0.2)   
    print("Val Score: ", model.evaluate(val_x, val_y))
    
    return results



model_history = [] 

for i in range(n_folds):
    print("Training on Fold: ",i+1)
    
    t_x, val_x, t_y, val_y = train_test_split(train_x, train_y ,test_size=0.2 ,shuffle=True)
    
    model_history.append(fit_and_evaluate(t_x, val_x, t_y, val_y, epochs, batch_size))
    
    model_  = load_model('subjek1CNN.h5')
    x_pred  = model_.predict_classes(val_x)
    y_pred  = np.argmax(val_y, axis=1)
    
    cm = confusion_matrix(x_pred, y_pred)
    print(classification_report(x_pred, y_pred))
    
    plt.figure(figsize = (5,5))
    sign = ["G1", "G2","G3","G4"]
    sns.heatmap(cm,cmap= "Blues",xticklabels=sign, yticklabels=sign, 
                linecolor = 'black' , linewidth = 1 , annot = True, fmt='')
    plt.show()
    
     

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)
fig, (ax1, ax2) =  plt.subplots( ncols=2, sharex=True)
ax1.plot(model_history[0].history['accuracy'], label='Training Fold 1 accuration')
ax1.plot(model_history[1].history['accuracy'], label='Training Fold 2 accuration')
ax1.plot(model_history[2].history['accuracy'], label='Training Fold 3 accuration')
ax1.plot(model_history[3].history['accuracy'], label='Training Fold 4 accuration')
ax1.plot(model_history[4].history['accuracy'], label='Training Fold 5 accuration')
ax1.legend()
ax2.plot(model_history[1].history['loss'], label='Training Fold 1 loss')
ax2.plot(model_history[1].history['loss'], label='Training Fold 2 loss')
ax2.plot(model_history[2].history['loss'], label='Training Fold 3 loss')
ax2.plot(model_history[3].history['loss'], label='Training Fold 4 loss ')
ax2.plot(model_history[4].history['loss'], label='Training Fold 5 loss')
ax2.legend()
plt.show()


In [ ]:
fig, (plt1, plt2)  =  plt.subplots( ncols=2, sharex=True)
plt1.plot(model_history[0].history['accuracy'], label='Train Accuracy Fold 1', color='black')
plt1.plot(model_history[0].history['val_accuracy'], label='Val Accuracy Fold 1', color='orange', linestyle = "dashdot")
plt1.legend()
plt2.plot(model_history[0].history['loss'], label='Train Loss Fold 1', color='black')
plt2.plot(model_history[0].history['val_loss'], label='Val Loss Fold 1', color='orange', linestyle = "dashdot")
plt2.legend()
plt.show()

fig, (plt1, plt2)  =  plt.subplots( ncols=2, sharex=True)
plt1.plot(model_history[1].history['accuracy'], label='Train Accuracy Fold 2', color='red')
plt1.plot(model_history[1].history['val_accuracy'], label='Val Accuracy Fold 2', color='orange', linestyle = "dashdot")
plt1.legend()
plt2.plot(model_history[1].history['loss'], label='Train Loss Fold 2', color='red')
plt2.plot(model_history[1].history['val_loss'], label='Val Loss Fold 2', color='orange', linestyle = "dashdot")
plt2.legend()
plt.show()

fig, (plt1, plt2)  =  plt.subplots( ncols=2, sharex=True)
plt1.plot(model_history[2].history['accuracy'], label='Train Accuracy Fold 3', color='green')
plt1.plot(model_history[2].history['val_accuracy'], label='Val Accuracy Fold 3', color='orange', linestyle = "dashdot")
plt1.legend()
plt2.plot(model_history[2].history['loss'], label='Train Loss Fold 3', color='green')
plt2.plot(model_history[2].history['val_loss'], label='Val Loss Fold 3', color='orange', linestyle = "dashdot")
plt2.legend()
plt.show()


fig, (plt1, plt2)  =  plt.subplots( ncols=2, sharex=True)
plt1.plot(model_history[3].history['accuracy'], label='Train Accuracy Fold 4', color='blue')
plt1.plot(model_history[3].history['val_accuracy'], label='Val Accuracy Fold 4', color='orange', linestyle = "dashdot")
plt1.legend()
plt2.plot(model_history[3].history['loss'], label='Train Loss Fold 4', color='blue')
plt2.plot(model_history[3].history['val_loss'], label='Val Loss Fold 4', color='orange', linestyle = "dashdot")
plt2.legend()
plt.show()

fig, (plt1, plt2)  =  plt.subplots( ncols=2, sharex=True)
plt1.plot(model_history[4].history['accuracy'], label='Train Accuracy Fold 5', color='purple')
plt1.plot(model_history[4].history['val_accuracy'], label='Val Accuracy Fold 5', color='orange', linestyle = "dashdot")
plt1.legend()
plt2.plot(model_history[4].history['loss'], label='Train Loss Fold 4', color='purple')
plt2.plot(model_history[4].history['val_loss'], label='Val Loss Fold 4', color='orange', linestyle = "dashdot")
plt2.legend()
plt.show()

In [ ]:
model =load_model('cnn.h5')
a = model.evaluate(test_x, test_y)